In [62]:
import tensorflow

from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import tensorflow as tf
import random
log_files_path = 'Data/Wavelet/Merged/'

#iloc makes sure we exclude our dependent variables/output
df=pd.read_csv(log_files_path + 'denoised2level2merged.csv').iloc[:,4:]
d = df.values
ntotal=len(df)

d = normalize(d, axis=0, norm='l2')

resultu = []
np.random.rand(4)
# Return 100 results (for instance)
for i in range(ntotal):
    
    res = random.random()
    if res < 0.1:
        resultu.append(1)
    elif res < 0.2 and res>=0.1:
        resultu.append(2)
    elif res < 0.3 and res>=0.2:
        resultu.append(3)
    elif res < 0.4 and res>=0.3:
        resultu.append(4)
    elif res < 0.5 and res>=0.4:
        resultu.append(5)
    elif res < 0.6 and res>=0.5:
        resultu.append(6)
    elif res < 0.7 and res>=0.6:
        resultu.append(8)
    elif res < 0.8 and res>=0.7:
        resultu.append(8)
    else:
        resultu.append(9)
resultu=np.array(resultu)
trainset=[]
for i in range(1,7):
    toinsert=d[resultu==i].astype(np.float32)
    trainset.append(toinsert)
validationset=d[resultu==8].astype(np.float32)
testset=d[resultu==9].astype(np.float32)


#x = data

# Following Hinton-Salakhutdinov Architecture

n_code_1 = 17
n_code_2 = 13
n_code_3 = 9

# Parameters
learning_rate = 0.005

#batch_size = 7
display_step = 1

total_batch = 6
training_epochs = 5000

In [63]:
#we store the variables here
log_files_path = 'Output/WT3layer17-13-9/'
input_size = df.shape[1]
def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    
    
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    
    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
        
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))
    
    
    #Here, we changed the shape of x into [[[x1]],[[x2] ],....]

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])

def layer(x, weight_shape, bias_shape, phase_train):
    
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(x, W) + b
    
    #apply the non-linear function after the batch normalization
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))
def encoder(x, n_code_1, n_code_2, n_code_3, phase_train):
    
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    with tf.variable_scope("encoder1", reuse=tf.AUTO_REUSE):
        encoder1 = layer(x, [input_size, n_code_1], [n_code_1], phase_train)
 
    with tf.variable_scope("encoder2", reuse=tf.AUTO_REUSE):
        encoder2 = layer(encoder1, [n_code_1, n_code_2], [n_code_2], phase_train)
    
    with tf.variable_scope("encoder3", reuse=tf.AUTO_REUSE):
        output = layer(encoder2, [n_code_2, n_code_3], [n_code_3], phase_train)

    
    return output


def decoder(x, n_code_1, n_code_2, n_code_3, phase_train):
    """
    Defines the network encoder part
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    

    with tf.variable_scope("decoder1", reuse=tf.AUTO_REUSE):
        decoder1 = layer(x, [n_code_3, n_code_2], [n_code_2], phase_train)
        
    with tf.variable_scope("decoder2", reuse=tf.AUTO_REUSE):
        decoder2 = layer(decoder1, [n_code_2, n_code_1], [n_code_1], phase_train)
    
    with tf.variable_scope("decoder3", reuse=tf.AUTO_REUSE):
        output = layer(decoder2, [n_code_1, input_size], [input_size], phase_train)
        
    return output


def loss(output, x):
    """
    Compute the loss of the auto-encoder
    
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    
    with tf.variable_scope("training", reuse=tf.AUTO_REUSE):
        
        l2 = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op
    
    
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op
def evaluate(output, x):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -x: true value for the validation set
    output:
        - val_loss: loss of the autoencoder
        - in_image_op: input image 
        - out_image_op:reconstructed image 
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        #in_image_op = image_summary("input_image", x)
        
        #out_image_op = image_summary("output_image", output)
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        #return val_loss, in_image_op, out_image_op, val_summary_op
        return val_loss,  val_summary_op
"""
def image_summary(label, tensor):
    #tf.summary.image: https://www.tensorflow.org/api_docs/python/tf/summary/image
    #Outputs a Summary protocol buffer with images.

    tensor_reshaped = tf.reshape(tensor, [-1, 19, 1, 1])
    return tf.summary.image(label, tensor_reshaped)
"""
if __name__ == '__main__':
    print('we begin')
    
    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            # image vector & label, phase_train is a boolean 
            x = tf.placeholder("float", [None, input_size])
            
            phase_train = tf.placeholder(tf.bool)
            
            #define the encoder 
            code = encoder(x, n_code_1, n_code_2, n_code_3, phase_train)
            
            #define the decoder
            output = decoder(code, n_code_1, n_code_2, n_code_3, phase_train)
            
            #compute the loss 
            cost, train_summary_op = loss(output, x)

            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            #evaluate the accuracy of the network (done on a validation set)
            #eval_op, in_image_op, out_image_op, val_summary_op = evaluate(output, x)
            eval_op, val_summary_op = evaluate(output, x)

            summary_op = tf.summary.merge_all()

            #save and restore variables to and from checkpoints.
            #saver = tf.train.Saver(max_to_keep=200)
            saver = tf.train.Saver()

            #defines a session
            sess = tf.Session()

            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            #train_writer = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #val_writer   = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)
            currentmin=10000000000
            currentmin_index=-1
            error_path=[]
            togive2=[]
            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                #total_batch = int(numberofsamples/batch_size)
                
                # Loop over all batches
                for i in range(total_batch):
                    
                    minibatch_x= trainset[i]
                    
                    # Fit training using batch data
                    #the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, phase_train: True})
                    
                    #train_writer.add_summary(train_summary, sess.run(global_step))
                    
                    # Compute average loss
                    avg_cost += new_cost/total_batch
                    
                
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost =", "{:0.9f}".format(avg_cost))

                    #the accuracy is evaluated using the validation dataset
                    #train_writer.add_summary(train_summary, sess.run(global_step))

                    #validation_loss, in_image, out_image, val_summary = sess.run([eval_op, in_image_op, out_image_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    validation_loss,  val_summary = sess.run([eval_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    #val_writer.add_summary(in_image, sess.run(global_step))
                    #val_writer.add_summary(out_image, sess.run(global_step))
                    #val_writer.add_summary(val_summary, sess.run(global_step))
                    print("Validation Loss:", validation_loss)
                    error_path.append(validation_loss)
                    
                    if validation_loss<currentmin:
                        currentmin=validation_loss
                        currentmin_index=epoch
                        saver.save(sess, log_files_path + 'tmp/model.ckpt')
                        togive8=[]
                        for i in range(len(d)):
                            any_image = d[i].reshape(-1,input_size)
                            output_any_image = sess.run(code,feed_dict={x:any_image,phase_train: False})
                            togive8.append(output_any_image)
                            
                        togive2=togive8
                    #save to use later
                    #https://www.tensorflow.org/api_docs/python/tf/train/Saver
                    #saver.save(sess, log_files_path+'model-checkpoint', global_step=global_step)
                    #saver.save(sess, 'C:/Users/yy2895/Desktop/st19-14-19/tmp/model.ckpt')
                    

            print("Optimization Finished!")

            test_loss = sess.run(eval_op, feed_dict={x: testset, phase_train: False})
            #nps_loss = sess.run(eval_op, feed_dict={x: mnist.validation.images, phase_train: False})
            print("Test Loss:", test_loss)
            

we begin
Epoch: 0001 cost = 2.296026349
Validation Loss: 2.907045
Epoch: 0002 cost = 2.275152008
Validation Loss: 2.8725493
Epoch: 0003 cost = 2.252218207
Validation Loss: 2.5852115
Epoch: 0004 cost = 2.226364175
Validation Loss: 2.3727944
Epoch: 0005 cost = 2.199892282
Validation Loss: 2.2168288
Epoch: 0006 cost = 2.175241629
Validation Loss: 2.134807
Epoch: 0007 cost = 2.150860469
Validation Loss: 2.06283
Epoch: 0008 cost = 2.128660162
Validation Loss: 2.0463817
Epoch: 0009 cost = 2.108073235
Validation Loss: 2.0131505
Epoch: 0010 cost = 2.088161389
Validation Loss: 1.9513338
Epoch: 0011 cost = 2.067885518
Validation Loss: 1.820314
Epoch: 0012 cost = 2.047118147
Validation Loss: 1.8621129
Epoch: 0013 cost = 2.026631951
Validation Loss: 1.9211538
Epoch: 0014 cost = 2.006992459
Validation Loss: 1.9860094
Epoch: 0015 cost = 1.989963373
Validation Loss: 1.9197644
Epoch: 0016 cost = 1.976218482
Validation Loss: 1.8175762
Epoch: 0017 cost = 1.963478724
Validation Loss: 2.10101
Epoch: 0018 

Validation Loss: 0.9392104
Epoch: 0151 cost = 1.081083834
Validation Loss: 1.0175322
Epoch: 0152 cost = 1.076909880
Validation Loss: 0.7617051
Epoch: 0153 cost = 1.071749767
Validation Loss: 0.8340185
Epoch: 0154 cost = 1.067194402
Validation Loss: 0.9054428
Epoch: 0155 cost = 1.063811660
Validation Loss: 1.1406349
Epoch: 0156 cost = 1.059458733
Validation Loss: 0.87004954
Epoch: 0157 cost = 1.054612597
Validation Loss: 0.71636003
Epoch: 0158 cost = 1.051040947
Validation Loss: 0.9260978
Epoch: 0159 cost = 1.046514908
Validation Loss: 0.94455373
Epoch: 0160 cost = 1.041829725
Validation Loss: 0.9320666
Epoch: 0161 cost = 1.037855784
Validation Loss: 0.6704931
Epoch: 0162 cost = 1.033553918
Validation Loss: 1.3513855
Epoch: 0163 cost = 1.029735466
Validation Loss: 0.95405567
Epoch: 0164 cost = 1.025516888
Validation Loss: 0.9223707
Epoch: 0165 cost = 1.021183252
Validation Loss: 0.819731
Epoch: 0166 cost = 1.017346859
Validation Loss: 0.7282803
Epoch: 0167 cost = 1.013230443
Validation 

Validation Loss: 0.73427474
Epoch: 0300 cost = 0.641326874
Validation Loss: 0.6357179
Epoch: 0301 cost = 0.639393896
Validation Loss: 0.61000067
Epoch: 0302 cost = 0.637382686
Validation Loss: 0.6344888
Epoch: 0303 cost = 0.635652920
Validation Loss: 0.74981606
Epoch: 0304 cost = 0.633708964
Validation Loss: 0.8913997
Epoch: 0305 cost = 0.631826073
Validation Loss: 0.8756181
Epoch: 0306 cost = 0.630118807
Validation Loss: 0.72676975
Epoch: 0307 cost = 0.628180703
Validation Loss: 0.6190647
Epoch: 0308 cost = 0.626343966
Validation Loss: 0.6175
Epoch: 0309 cost = 0.624623458
Validation Loss: 0.5387815
Epoch: 0310 cost = 0.622722973
Validation Loss: 0.5689607
Epoch: 0311 cost = 0.620962997
Validation Loss: 0.5826627
Epoch: 0312 cost = 0.619256794
Validation Loss: 0.5592589
Epoch: 0313 cost = 0.617387603
Validation Loss: 0.5889532
Epoch: 0314 cost = 0.615646889
Validation Loss: 0.64571244
Epoch: 0315 cost = 0.613948007
Validation Loss: 0.6238583
Epoch: 0316 cost = 0.612125983
Validation L

Epoch: 0445 cost = 0.448824719
Validation Loss: 0.5842403
Epoch: 0446 cost = 0.447886993
Validation Loss: 0.53381264
Epoch: 0447 cost = 0.446907202
Validation Loss: 0.47960857
Epoch: 0448 cost = 0.446078892
Validation Loss: 0.4366995
Epoch: 0449 cost = 0.445186789
Validation Loss: 0.40236875
Epoch: 0450 cost = 0.444245845
Validation Loss: 0.40925568
Epoch: 0451 cost = 0.443428884
Validation Loss: 0.43327498
Epoch: 0452 cost = 0.442549100
Validation Loss: 0.4110872
Epoch: 0453 cost = 0.441594407
Validation Loss: 0.39356333
Epoch: 0454 cost = 0.440762733
Validation Loss: 0.378618
Epoch: 0455 cost = 0.439929128
Validation Loss: 0.40301666
Epoch: 0456 cost = 0.439011986
Validation Loss: 0.40720257
Epoch: 0457 cost = 0.438183432
Validation Loss: 0.38524002
Epoch: 0458 cost = 0.437375610
Validation Loss: 0.37999058
Epoch: 0459 cost = 0.436457838
Validation Loss: 0.46541834
Epoch: 0460 cost = 0.435595681
Validation Loss: 0.53498197
Epoch: 0461 cost = 0.434818357
Validation Loss: 0.5022084
Epo

Epoch: 0587 cost = 0.354737982
Validation Loss: 0.33003873
Epoch: 0588 cost = 0.354272758
Validation Loss: 0.33578858
Epoch: 0589 cost = 0.353719125
Validation Loss: 0.35749644
Epoch: 0590 cost = 0.353299797
Validation Loss: 0.3519792
Epoch: 0591 cost = 0.352906406
Validation Loss: 0.32453305
Epoch: 0592 cost = 0.352379233
Validation Loss: 0.33009803
Epoch: 0593 cost = 0.351914947
Validation Loss: 0.34526002
Epoch: 0594 cost = 0.351554334
Validation Loss: 0.34757632
Epoch: 0595 cost = 0.351044953
Validation Loss: 0.3468862
Epoch: 0596 cost = 0.350519692
Validation Loss: 0.34610873
Epoch: 0597 cost = 0.350174497
Validation Loss: 0.34677884
Epoch: 0598 cost = 0.349762122
Validation Loss: 0.39483267
Epoch: 0599 cost = 0.349215110
Validation Loss: 0.34397018
Epoch: 0600 cost = 0.348827899
Validation Loss: 0.38568237
Epoch: 0601 cost = 0.348480279
Validation Loss: 0.38172516
Epoch: 0602 cost = 0.347915376
Validation Loss: 0.3311706
Epoch: 0603 cost = 0.347458879
Validation Loss: 0.33200866


...

Epoch: 4434 cost = 0.024708277
Validation Loss: 0.07932891
Epoch: 4435 cost = 0.024729610
Validation Loss: 0.031412754
Epoch: 4436 cost = 0.024774203
Validation Loss: 0.027685536
Epoch: 4437 cost = 0.024712321
Validation Loss: 0.024984032
Epoch: 4438 cost = 0.024738108
Validation Loss: 0.026932085
Epoch: 4439 cost = 0.024729756
Validation Loss: 0.027780809
Epoch: 4440 cost = 0.024721462
Validation Loss: 0.027493836
Epoch: 4441 cost = 0.024699466
Validation Loss: 0.026113681
Epoch: 4442 cost = 0.024724289
Validation Loss: 0.02604087
Epoch: 4443 cost = 0.024762847
Validation Loss: 0.028168475
Epoch: 4444 cost = 0.024702395
Validation Loss: 0.027801266
Epoch: 4445 cost = 0.024731207
Validation Loss: 0.025867738
Epoch: 4446 cost = 0.024721224
Validation Loss: 0.025081666
Epoch: 4447 cost = 0.024712981
Validation Loss: 0.02538422
Epoch: 4448 cost = 0.024691164
Validation Loss: 0.025289068
Epoch: 4449 cost = 0.024720213
Validation Loss: 0.026616827
Epoch: 4450 cost = 0.024750681
Validation L

Epoch: 4578 cost = 0.024597749
Validation Loss: 0.025530603
Epoch: 4579 cost = 0.024607536
Validation Loss: 0.025234321
Epoch: 4580 cost = 0.024533440
Validation Loss: 0.025649443
Epoch: 4581 cost = 0.024523407
Validation Loss: 0.027648773
Epoch: 4582 cost = 0.024566977
Validation Loss: 0.034991477
Epoch: 4583 cost = 0.024634120
Validation Loss: 0.025303576
Epoch: 4584 cost = 0.024578446
Validation Loss: 0.027487682
Epoch: 4585 cost = 0.024613385
Validation Loss: 0.025801146
Epoch: 4586 cost = 0.024620020
Validation Loss: 0.029411089
Epoch: 4587 cost = 0.024581941
Validation Loss: 0.024246635
Epoch: 4588 cost = 0.024572657
Validation Loss: 0.033009052
Epoch: 4589 cost = 0.024542099
Validation Loss: 0.02924553
Epoch: 4590 cost = 0.024540043
Validation Loss: 0.02689288
Epoch: 4591 cost = 0.024582952
Validation Loss: 0.024253724
Epoch: 4592 cost = 0.024592565
Validation Loss: 0.024743255
Epoch: 4593 cost = 0.024511854
Validation Loss: 0.024762455
Epoch: 4594 cost = 0.024515453
Validation 

Epoch: 4718 cost = 0.024421096
Validation Loss: 0.025879184
Epoch: 4719 cost = 0.024466164
Validation Loss: 0.02918781
Epoch: 4720 cost = 0.024455968
Validation Loss: 0.029484961
Epoch: 4721 cost = 0.024481682
Validation Loss: 0.026528277
Epoch: 4722 cost = 0.024452730
Validation Loss: 0.027369985
Epoch: 4723 cost = 0.024455442
Validation Loss: 0.025350612
Epoch: 4724 cost = 0.024469281
Validation Loss: 0.026037911
Epoch: 4725 cost = 0.024496570
Validation Loss: 0.025629593
Epoch: 4726 cost = 0.024447865
Validation Loss: 0.027026543
Epoch: 4727 cost = 0.024475805
Validation Loss: 0.025288373
Epoch: 4728 cost = 0.024462561
Validation Loss: 0.028025178
Epoch: 4729 cost = 0.024441685
Validation Loss: 0.025911698
Epoch: 4730 cost = 0.024448139
Validation Loss: 0.025331499
Epoch: 4731 cost = 0.024519031
Validation Loss: 0.025350956
Epoch: 4732 cost = 0.024520542
Validation Loss: 0.028598648
Epoch: 4733 cost = 0.024485317
Validation Loss: 0.027892508
Epoch: 4734 cost = 0.024525060
Validation

Epoch: 4856 cost = 0.024335040
Validation Loss: 0.025725128
Epoch: 4857 cost = 0.024308902
Validation Loss: 0.028301537
Epoch: 4858 cost = 0.024319628
Validation Loss: 0.024798363
Epoch: 4859 cost = 0.024374561
Validation Loss: 0.02379994
Epoch: 4860 cost = 0.024303705
Validation Loss: 0.02587782
Epoch: 4861 cost = 0.024295849
Validation Loss: 0.026156701
Epoch: 4862 cost = 0.024295797
Validation Loss: 0.028737023
Epoch: 4863 cost = 0.024315862
Validation Loss: 0.03145625
Epoch: 4864 cost = 0.024276736
Validation Loss: 0.030122563
Epoch: 4865 cost = 0.024318925
Validation Loss: 0.023991775
Epoch: 4866 cost = 0.024290935
Validation Loss: 0.02518401
Epoch: 4867 cost = 0.024259135
Validation Loss: 0.025834126
Epoch: 4868 cost = 0.024264223
Validation Loss: 0.026671316
Epoch: 4869 cost = 0.024271686
Validation Loss: 0.026977073
Epoch: 4870 cost = 0.024259177
Validation Loss: 0.02600527
Epoch: 4871 cost = 0.024327186
Validation Loss: 0.024253344
Epoch: 4872 cost = 0.024364988
Validation Los

Epoch: 5000 cost = 0.024308184
Validation Loss: 0.029326448
Optimization Finished!
Test Loss: 0.024569033


In [64]:
import csv
with open(log_files_path + '3layer17-13-9.csv', 'w',newline='') as myfile:
     wr = csv.writer(myfile)
     for i in range(len(togive2)):
        wr.writerow(list(togive2[i][0]))